In [1]:
# 01_smoke_test.ipynb
# End-to-end sanity check for backtest_engine

from datetime import date
from pathlib import Path

import pandas as pd

from backtest_engine import BacktestEngine
from backtest_engine.config import (
    BacktestConfig,
    StudyWindow,
    UniverseConfig,
    SignalConfig,
    SelectionConfig,
    RebalanceConfig,
    FeeConfig,
)
from backtest_engine.postgres_provider import PostgresDataProvider
from backtest_engine.db import run_sql

pd.options.display.max_rows = 20
pd.options.display.max_columns = None
pd.options.display.width = 140

print("✅ Imports OK")

import os

# Your usual DB settings
os.environ["PGHOST"] = "localhost"
os.environ["PGPORT"] = "5433"
os.environ["PGDATABASE"] = "kairo_production"
os.environ["PGUSER"] = "sanjay_readonly"
os.environ["PGPASSWORD"] = "Piper112358!"

✅ Imports OK


In [2]:
# Quick DB check – requires tunnel + env vars set in terminal
# (PGHOST, PGPORT, PGDATABASE, PGUSER, PGPASSWORD)

try:
    df_now = run_sql("SELECT now() AT TIME ZONE 'Asia/Kolkata' AS ist_now;")
    display(df_now)
    print("✅ DB connected and responding")
except Exception as e:
    print("❌ DB connectivity issue:", repr(e))
    print("Check SSH tunnel + PG* environment variables before running backtests.")

,ist_now
0,2025-11-28 10:46:56.131529


✅ DB connected and responding


In [3]:
from datetime import date
from pathlib import Path

from backtest_engine import run_batch
from backtest_engine.postgres_provider import PostgresDataProvider
from backtest_engine.config import (
    BacktestConfig,
    StudyWindow,
    UniverseConfig,
    SignalConfig,
    SelectionConfig,
    RebalanceConfig,
    FeeConfig,
)

provider = PostgresDataProvider()

base_window = StudyWindow(start=date(2014, 1, 1), end=date(2024, 1, 1))

configs = [
    BacktestConfig(
        name="mf-10y-annual-top15",
        study_window=base_window,
        universe=UniverseConfig(preset="equity_active_direct"),
        signal=SignalConfig(
            name="rank_12m_category",
            direction="asc",
        ),
        selection=SelectionConfig(
            mode="top_n",
            top_n=15,
            min_funds=10,
            weight_scheme="equal",
        ),
        rebalance=RebalanceConfig(frequency="12M"),
        fees=FeeConfig(apply=True, annual_bps=100.0),
    ),
    BacktestConfig(
        name="mf-10y-annual-top30",
        study_window=base_window,
        universe=UniverseConfig(preset="equity_active_direct"),
        signal=SignalConfig(
            name="rank_12m_category",
            direction="asc",
        ),
        selection=SelectionConfig(
            mode="top_n",
            top_n=30,
            min_funds=20,
            weight_scheme="equal",
        ),
        rebalance=RebalanceConfig(frequency="12M"),
        fees=FeeConfig(apply=True, annual_bps=100.0),
    ),
]

batch_out_dir = Path("batch_outputs")

batch_summary = run_batch(
    configs=configs,
    data_provider=provider,
    out_dir=batch_out_dir,
    level="standard",  # or "light" / "full"
)

batch_summary


,run_id,name,start_date,end_date,num_periods,gross_return,gross_cagr,net_return,net_cagr,benchmark_return,benchmark_cagr,alpha_return,alpha_cagr,net_alpha_return,net_alpha_cagr,fees_applied,fees_annual_bps,output_dir,output_files
0,mf-10y-annual-top15,mf-10y-annual-top15,2014-01-01,2024-01-01,10,4.806077,0.192193,4.250623,0.180271,2.872259,0.144891,1.933818,0.047301,1.378364,0.035379,True,100.0,batch_outputs/mf-10y-annual-top15,batch_outputs/mf-10y-annual-top15/mf-10y-annua...
1,mf-10y-annual-top30,mf-10y-annual-top30,2014-01-01,2024-01-01,10,4.520314,0.186194,3.992198,0.174332,2.872259,0.144891,1.648055,0.041303,1.119939,0.029441,True,100.0,batch_outputs/mf-10y-annual-top30,batch_outputs/mf-10y-annual-top30/mf-10y-annua...


In [ ]:
# Period-level view: one row per rebalance period

display(
    result.portfolio_periods[
        [
            "period_no",
            "start_date",
            "end_date",
            "period_days",
            "num_funds",
            "gross_return",
            "net_return",
            "benchmark_return",
            "alpha_return",
            "net_alpha_return",
        ]
    ]
    .head(10)
)

# Holdings view: first few rows of fund-level detail
display(
    result.holdings[
        [
            "run_id",
            "period_no",
            "rebalance_date",
            "schemecode",
            "scheme_name",
            "weight",
            "fund_return",
            "period_gross_return",
            "period_net_return",
        ]
    ]
    .head(20)
)

,period_no,start_date,end_date,period_days,num_funds,gross_return,net_return,benchmark_return,alpha_return,net_alpha_return
0,1,2014-01-01,2015-01-01,365,15,0.510397,0.495293,0.378938,0.131459,0.116355
1,2,2015-01-01,2016-01-01,365,15,0.046460,0.035995,-0.004782,0.051242,0.040777
2,3,2016-01-01,2017-01-01,366,15,0.038411,0.027999,0.033930,0.004482,-0.005931
3,4,2017-01-01,2018-01-01,365,15,0.376957,0.363188,0.347305,0.029653,0.015883
4,5,2018-01-01,2019-01-01,365,15,-0.086565,-0.095700,-0.025078,-0.061487,-0.070621
5,6,2019-01-01,2020-01-01,365,15,0.088254,0.077371,0.075088,0.013166,0.002283
6,7,2020-01-01,2021-01-01,366,15,0.202399,0.190343,0.170531,0.031869,0.019812
7,8,2021-01-01,2022-01-01,365,15,0.475905,0.461146,0.295585,0.180320,0.165561
8,9,2022-01-01,2023-01-01,365,15,0.050889,0.040381,0.016064,0.034825,0.024316
9,10,2023-01-01,2024-01-01,365,15,0.385821,0.371963,0.254183,0.131638,0.117780


,run_id,period_no,rebalance_date,schemecode,scheme_name,weight,fund_return,period_gross_return,period_net_return
0,mf-10y-annual-rebal-net-fee,1,2014-01-01,18512,ICICI Pru Technology Fund(G)-Direct Plan,0.066667,0.281517,0.510397,0.495293
1,mf-10y-annual-rebal-net-fee,1,2014-01-01,19008,SBI Midcap Fund(G)-Direct Plan,0.066667,0.732402,0.510397,0.495293
2,mf-10y-annual-rebal-net-fee,1,2014-01-01,18465,HSBC Flexi Cap Fund(G)-Direct Plan,0.066667,0.560412,0.510397,0.495293
3,mf-10y-annual-rebal-net-fee,1,2014-01-01,18192,Axis ELSS Tax Saver Fund(G)-Direct Plan,0.066667,0.693876,0.510397,0.495293
4,mf-10y-annual-rebal-net-fee,1,2014-01-01,18146,Franklin India Focused Equity Fund(G)-Direct Plan,0.066667,0.806433,0.510397,0.495293
5,mf-10y-annual-rebal-net-fee,1,2014-01-01,18924,Invesco India Large & Mid Cap Fund(G)-Direct Plan,0.066667,0.457385,0.510397,0.495293
6,mf-10y-annual-rebal-net-fee,1,2014-01-01,18186,Axis Large Cap Fund(G)-Direct Plan,0.066667,0.426407,0.510397,0.495293
7,mf-10y-annual-rebal-net-fee,1,2014-01-01,18106,HDFC Capital Builder Value Fund(G)-Direct Plan,0.066667,0.528956,0.510397,0.495293
8,mf-10y-annual-rebal-net-fee,1,2014-01-01,18635,Kotak Contra Fund(G)-Direct Plan,0.066667,0.402472,0.510397,0.495293
9,mf-10y-annual-rebal-net-fee,1,2014-01-01,18134,Franklin India Small Cap Fund(G)-Direct Plan,0.066667,0.910110,0.510397,0.495293
